## Architecture
* Image: nfnet-l0 + Efficientnet-b5 (Arcface Module)
* Text : TfidfVectorizer + Sbert (Arcface Module)

In [1]:
import sys
sys.path.append('pytorch-image-models')

In [83]:
import numpy as np 
import pandas as pd 

import math
import random 
import os 
import cv2
import timm

from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset 
from torch import nn
from torch.nn import Conv2d, AdaptiveAvgPool2d, Linear
from torch.nn.parameter import Parameter
import torch.nn.functional as F 
from functools import reduce

import transformers

import gc
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [84]:
class CFG:
    # Image
    img_size = 512
    batch_size = 16
    seed = 42
    
    device = 'cuda'
    classes1 = 11014
    
    model_name1 = 'eca_nfnet_l0'
    model_path1 = '../input/shopee-pytorch-models/arcface_512x512_nfnet_l0 (mish).pt'
    model_name2 = 'tf_efficientnet_b5'
    model_path2 = '../input/shopee-hash-model-eff-b5/arcface_512x512_tf_efficientnet_b5(mish).pt'
    model_name3 = 'efficientnet_b5'
    model_path3 = '../input/shopee-pytorch-models/arcface_512x512_eff_b5_.pt'
    
    scale = 30 
    margin = 0.5
    
    # Text
    CV = False
    num_workers=4
    transformer_model = '../input/sentence-transformer-models/paraphrase-xlm-r-multilingual-v1/0_Transformer'
    text_model_path = '../input/best-multilingual-model/sentence_transfomer_xlm_best_loss_num_epochs_25_arcface.bin'
    
    model_params = {
    'n_classes':11014,
    'model_name':transformer_model,
    'use_fc':False,
    'fc_dim':512,
    'dropout':0.3,
    }

In [85]:
tokenizer = transformers.AutoTokenizer.from_pretrained(CFG.transformer_model)

In [87]:
def read_dataset(is_train=False):
    if is_train:
        df = pd.read_csv('../input/shopee-product-matching/train.csv')
        if CFG.CV:
            tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
            df['matches'] = df['label_group'].map(tmp)
            df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
        df_cu = cudf.DataFrame(df)
        image_paths = '../input/shopee-product-matching/train_images/' + df['image']
    else:
        df = pd.read_csv('../input/shopee-product-matching/test.csv')
        df_cu = cudf.DataFrame(df)
        image_paths = '../input/shopee-product-matching/test_images/' + df['image']
    return df, df_cu, image_paths

In [88]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

In [89]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions1'], row['image_predictions3'], row['text_predictions'], row['text_predictions2']])
    return ' '.join( np.unique(x))

def combine_for_cv(row):
    x = np.concatenate([row['image_predictions1'], row['image_predictions3'], row['text_predictions'], row['text_predictions2']])
    return np.unique(x)

In [90]:
def get_image_predictions(df, embeddings, threshold=0.0):
    if len(df) > 3:
        KNN = 50
    else : 
        KNN = 3
    
    model = NearestNeighbors(n_neighbors=KNN, metric='cosine')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = df['posting_id'].iloc[ids].values
        predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return predictions

In [91]:
def get_test_transforms():
    return A.Compose(
        [
            A.Resize(CFG.img_size,CFG.img_size,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [92]:
class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [93]:
class ShopeeTextDataset(Dataset):
    def __init__(self, csv):
        self.csv = csv.reset_index()

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        
        text = row.title
        
        text = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        input_ids = text['input_ids'][0]
        attention_mask = text['attention_mask'][0]  
        
        return input_ids, attention_mask
    
    
class ShopeeTextNet(nn.Module):

    def __init__(self,
                 n_classes,
                 model_name='bert-base-uncased',
                 use_fc=False,
                 fc_dim=512,
                 dropout=0.0):
        """
        :param n_classes:
        :param model_name: name of model from pretrainedmodels
            e.g. resnet50, resnext101_32x4d, pnasnet5large
        :param pooling: One of ('SPoC', 'MAC', 'RMAC', 'GeM', 'Rpool', 'Flatten', 'CompactBilinearPooling')
        :param loss_module: One of ('arcface', 'cosface', 'softmax')
        """
        super(ShopeeTextNet, self).__init__()

        self.transformer = transformers.AutoModel.from_pretrained(model_name)
        final_in_features = self.transformer.config.hidden_size
        
        self.use_fc = use_fc
    
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim


    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, input_ids,attention_mask):
        feature = self.extract_feat(input_ids,attention_mask)
        return F.normalize(feature)

    def extract_feat(self, input_ids,attention_mask):
        x = self.transformer(input_ids=input_ids,attention_mask=attention_mask)
        
        features = x[0]
        features = features[:,0,:]

        if self.use_fc:
            features = self.dropout(features)
            features = self.fc(features)
            features = self.bn(features)

        return features

In [94]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # cos(theta) & phi(theta)
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        # convert label to one-hot
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

class ShopeeModel(nn.Module):
    def __init__(
        self,
        n_classes = CFG.classes1,
        model_name = CFG.model_name1,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'tf_efficientnet_b5' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif 'eca_nfnet' in model_name:
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

In [95]:
class Mish_func(torch.autograd.Function):
    
    """from: https://github.com/tyunist/memory_efficient_mish_swish/blob/master/mish.py"""
    
    @staticmethod
    def forward(ctx, i):
        result = i * torch.tanh(F.softplus(i))
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
  
        v = 1. + i.exp()
        h = v.log() 
        grad_gh = 1./h.cosh().pow_(2) 

        # Note that grad_hv * grad_vx = sigmoid(x)
        #grad_hv = 1./v  
        #grad_vx = i.exp()
        
        grad_hx = i.sigmoid()

        grad_gx = grad_gh *  grad_hx #grad_hv * grad_vx 
        
        grad_f =  torch.tanh(F.softplus(i)) + i * grad_gx 
        
        return grad_output * grad_f 


class Mish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        pass
    def forward(self, input_tensor):
        return Mish_func.apply(input_tensor)


def replace_activations(model, existing_layer, new_layer):
    
    """A function for replacing existing activation layers"""
    
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = replace_activations(module, existing_layer, new_layer)

        if type(module) == existing_layer:
            layer_old = module
            layer_new = new_layer
            model._modules[name] = layer_new
    return model

In [96]:
def get_image_embeddings(image_paths, model_name=CFG.model_name1, model_path=CFG.model_path1, n_classes=CFG.classes1):
    embeds = []
    
    model = ShopeeModel(model_name=model_name, n_classes=n_classes)
    model.eval()
    
    if model_name == 'eca_nfnet_l0' or model_name == 'tf_efficientnet_b5':
        model = replace_activations(model, torch.nn.SiLU, Mish())
    model.load_state_dict(torch.load(model_path))
    model = model.to(CFG.device)
    

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [97]:
def get_text_predictions(df, max_features = 25_000, th=0.75):
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,] > th)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
    
    del model, text_embeddings
    gc.collect()
    return preds

In [98]:
def get_text_embeddings(df):
    embeds = []
    
    model = ShopeeTextNet(**CFG.model_params)
    model.eval()
    
    model.load_state_dict(dict(list(torch.load(CFG.text_model_path).items())[:-1]))
    model = model.to(CFG.device)

    text_dataset = ShopeeTextDataset(df)
    text_loader = torch.utils.data.DataLoader(
        text_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=CFG.num_workers
    )    
    
    with torch.no_grad():
        for input_ids, attention_mask in tqdm(text_loader): 
            input_ids = input_ids.cuda()
            attention_mask = attention_mask.cuda()
            feat = model(input_ids, attention_mask)
            text_embeddings = feat.detach().cpu().numpy()
            embeds.append(text_embeddings)
    
    
    del model
    text_embeddings = np.concatenate(embeds)
    print(f'Our text embeddings shape is {text_embeddings.shape}')
    del embeds
    gc.collect()
    return text_embeddings

In [99]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1
   

def getMetric(col):
    def f1score(row):
        n = len(np.intersect1d(row.target, row[col]))
        return 2*n / (len(row.target) + len(row[col]))
    return f1score

In [100]:
def get_neighbours_cos_sim(df,embeddings, threshold=0.6):
    '''
    When using cos_sim use normalized features else use normal features
    '''
    embeddings = cupy.array(embeddings)
    
    if CFG.CV:
        thresholds = list(np.arange(0.5,0.7,0.05))

        scores = []
        for threshold in thresholds:
            preds = []
            CHUNK = 1024*4

            print('Finding similar titles...for threshold :',threshold)
            CTS = len(embeddings)//CHUNK
            if len(embeddings)%CHUNK!=0: CTS += 1

            for j in range( CTS ):
                a = j*CHUNK
                b = (j+1)*CHUNK
                b = min(b,len(embeddings))

                cts = cupy.matmul(embeddings,embeddings[a:b].T).T

                for k in range(b-a):
                    IDX = cupy.where(cts[k,]>threshold)[0]
                    o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
                    o = ' '.join(o)
                    preds.append(o)
                    
            df['pred_matches'] = preds
            df['f1'] = f1_score(df['matches'], df['pred_matches'])
            score = df['f1'].mean()
            print(f'Our f1 score for threshold {threshold} is {score}')
            scores.append(score)
            
        thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
        max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
        best_threshold = max_score['thresholds'].values[0]
        best_score = max_score['scores'].values[0]
        print(f'Our best score is {best_score} and has a threshold {best_threshold}')
            
    else:
        preds = []
        CHUNK = 1024*4

        print('Finding similar texts...for threshold :',threshold)
        CTS = len(embeddings)//CHUNK
        if len(embeddings)%CHUNK!=0: CTS += 1

        for j in range( CTS ):
            a = j*CHUNK
            b = (j+1)*CHUNK
            b = min(b,len(embeddings))
            print('chunk',a,'to',b)

            cts = cupy.matmul(embeddings,embeddings[a:b].T).T

            for k in range(b-a):
                IDX = cupy.where(cts[k,]>threshold)[0]
                o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
                preds.append(o)
                    
    return df, preds

In [108]:
def intersect(*args):
    return reduce(np.intersect1d, args)

def concat(*args):
    return np.unique(np.concatenate(args))

def higher(row):
    res = {}
    context = np.concatenate([row['img1'], row['img3'], row['text'], row['text2']])
    keys = np.unique(context)
    for k in keys: 
        res[k] = np.count_nonzero(context == k)
    output_dict = dict(filter(lambda item: item[1] >= 2, res.items()))
    
    return ' '.join((list(output_dict.keys())))

def count(row):
    res = {}
    context = np.concatenate([row['img1'], row['img3'], row['text'], row['text2']])
    keys = np.unique(context)
    for k in keys: 
        res[k] = np.count_nonzero(context == k)
    return res

In [101]:
df,df_cu,image_paths = read_dataset()
df.head()

,posting_id,image,image_phash,title
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng


In [102]:
image_embeddings = get_image_embeddings(image_paths.values, model_name=CFG.model_name1, model_path=CFG.model_path1, n_classes=CFG.classes1)

Building Model Backbone for eca_nfnet_l0 model


100%|██████████| 1/1 [00:00<00:00,  1.89it/s]

Our image embeddings shape is (3, 512)


In [103]:
#image_embeddings2 = get_image_embeddings(image_paths.values, model_name=CFG.model_name2, model_path=CFG.model_path2, n_classes=CFG.classes2)

In [104]:
image_embeddings3 = get_image_embeddings(image_paths.values, model_name=CFG.model_name3, model_path=CFG.model_path3, n_classes=CFG.classes1)

Building Model Backbone for efficientnet_b5 model


100%|██████████| 1/1 [00:00<00:00,  1.95it/s]

Our image embeddings shape is (3, 512)


In [105]:
image_predictions = get_image_predictions(df, image_embeddings, threshold = 0.33)
#image_predictions2 = get_image_predictions(df, image_embeddings2, threshold = 0.33)
image_predictions3 = get_image_predictions(df, image_embeddings3, threshold = 0.33)
text_predictions = get_text_predictions(df, max_features = 25_000)

100%|██████████| 3/3 [00:00<00:00, 3233.85it/s]


Finding similar titles...
chunk 0 to 3


In [106]:
text_embeddings = get_text_embeddings(df)

100%|██████████| 1/1 [00:00<00:00,  2.18it/s]


Our text embeddings shape is (3, 768)


In [107]:
df, text_predictions2 = get_neighbours_cos_sim(df, text_embeddings)

Finding similar texts...for threshold : 0.6
chunk 0 to 3


In [109]:
df['matches'] = pd.DataFrame({'img1': image_predictions,
                              'img3': image_predictions3,
                              'text': text_predictions,
                              'text2': text_predictions2}).apply(higher, axis=1)

In [110]:
df[['posting_id', 'matches']].to_csv('submission.csv', index = False)

In [111]:
#df['text_predictions'] = text_predictions
#df['image_predictions'] = image_predictions2
#df['matches_CV'] = df.apply(combine_for_cv, axis=1)
#tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
#df['target'] = df.label_group.map(tmp)
#MyCVScore = df.apply(getMetric('matches_CV'), axis=1)
#print('CV score =', MyCVScore.mean())

* efficientnet-b5 + tfidf
* CV score = 0.8983709554601149

* ensemble (nfnet_l0 + efficientnet-b5 + tfidf)
- CV score = 0.8969412521795675

* ensemble (nfnet_l0 + tf_eff_b5 + tfidf) using Count >= 2
- CV score = 0.897